# Model Explainations
## Questions to address:
- Which features are most important for the autoencoder's reconstruction?
(SHAP, LIME)
- Are there any specific feature interactions that affect the reconstruction quality? (SHAP dependence plots)
 -How do individual features affect the reconstruction error? (PDPs)
- Are there any clusters or patterns in the latent space? (Latent space visualization)
- What are the most informative dimensions of the latent space? (Supervised model on latent space)
- Are there any data points with unusually high reconstruction errors? (Reconstruction error analysis)
- Does the autoencoder effectively capture the relationships between the original features? (Feature correlation comparison)
- What are some of the features that are poorly reconstructed? (Original vs Reconstructed plots)
- Are there any noticeable changes in the distribution of the features after they have been reconstructed? (Distribution plots of original vs reconstructed data)

### Architecture Review:
Here's a breakdown of your autoencoder's architecture:
1. __Overall Structure:__
    - It's a feedforward neural network-based autoencoder.
    - The primary goal is to compress the input data (8 features) into a lower-dimensional latent space (`encoding_dim`) and then reconstruct a single output value ("Hydrogen Evolution Rate").
    - The architecture includes a custom attention mechanism.
2. __Encoder:__
    - Layers:
        - `nn.Linear(input_dim, 128)`:Maps the input (8 features) to 128 neurons.
        - `nn.Linear(128, 64)`:Maps 128 neurons to 64 neurons.
        - `nn.Linear(64, encoding_dim)`: Maps 64 neurons to the latent space of `encoding_dim` dimensions.
    - Activation Function: ReLU (Rectified Linear Unit) is used after each linear layer.
        - ReLU is computationally efficient and helps mitigate the vanishing gradient problem.
    - Function: The encoder compresses the input data into a lower-dimensional representation, capturing the most salient features.
3. __`Attention Mechanism (Attention class):`__
    - Purpose: To weigh the importance of different dimensions in the encoded representation.
    - Layers:
        - `nn.Linear(input_dim, input_dim)`: Transforms the input.  In this case, the input dim is the encoding dim.
        - `nn.Tanh()`: Applies the hyperbolic tangent activation function.
        - `nn.Linear(input_dim, 1)`: Maps each dimension to a single attention score.
        - `nn.Softmax(dim=1)`: Normalizes the attention scores across the `encoding_dim` to produce weights between 0 and 1 that sum to 1.
    - Operation:
        1. The attention network takes the encoded vector as input.
        2. It calculates attention weights for each dimension of the encoded vector.
        3. It multiplies the encoded vector by these weights.
    - Integration: The output of the attention mechanism is added to the original encoded vector (residual connection).
    - Impact: This mechanism allows the model to dynamically adjust the importance of each feature in the encoded representation before passing it to the decoder. This could help the model focus on the most relevant encoded features for reconstructing "Bubble Coverage".
4. __Decoder:__
    - Layers:
        - `nn.Linear(encoding_dim, 64)`: Maps the latent space to 64 neurons.
        - `nn.Linear(64, 128)`: Maps 64 neurons to 128 neurons.
        - `nn.Linear(128, 1)`: Maps 128 neurons to a single output value.
    - Activation Function: ReLU is used after the first two linear layers. The final linear layer does not have an activation function, which is typical for a regression output.
    - Function: The decoder takes the (weighted) encoded representation and expands it back to the output dimension (1, representing "Hydrogen Evolution Rate").
6. __Residual Connection:__
    - The output of the attention mechanism is added to the output of the encoder before being passed to the decoder.
    - Benefits:
        - Can help with gradient flow during training, especially in deeper networks.
        - Allows the model to learn both the direct mapping from the encoded features and the refined mapping learned by the attention mechanism.